# Practical Exam: Sample SQL Associate

Tech Solutions Inc. is a leading technology company specializing in software development and IT consulting services. The company prides itself on delivering innovative solutions to clients across various industries. With a dedicated team of skilled professionals, TechSolutions has earned a reputation for excellence in the tech industry.

Tech Solutions Inc. has been experiencing a decline in customer satisfaction ratings over the past few months. Customer feedback surveys and support tickets indicate an increase in dissatisfaction among clients. The company is concerned about this trend as it directly impacts customer retention, reputation, and overall business growth.

You are working with the customer support team to provide data to managers to help the company take proactive measures to address these concerns effectively. 

## Data

The following schema diagram shows the tables available.

![customer_feedback](support_db.png)

# Task 1

Before you can start any analysis, you need to confirm that the data is accurate and reflects what you expect to see. 

It is known that there are some issues with the `support` table, and the data team have provided the following data description. 

Write a query to return data matching this description. You must match all column names and description criteria.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|id | Discrete. The unique identifier of the support ticket. </br>Missing values are not possible due to the database structure.|
| customer_id | Discrete. The unique identifier of the customer. </br>Missing values should be replaced with 0.|
| category | Nominal. The gategory of the support request, can be one of Feedback, Billing Enquiry, Bug, Installation Problem, Other. </br>Missing values should be replaced with Other. |
| status | Nominal. The current status of the support ticket, one of Open, In Progress or Resolved. </br>Missing values should be replaced with 'Resolved'. |
| creation_date | Discrete. The date the ticket was created. Can be any date in 2023. </br>Missing values should be replaced with 2023-01-01. |
| response_time | Discrete. The number of days taken to respond to the support ticket. </br>Missing values should be replaced with 0. |
| resolution_time | Continuos. The number of hours taken to resolve the support ticket, rounded to 2 decimal places. </br>Missing values should be replaced with 0. |

In [75]:
-- Write your query for task 1 in this cell
SELECT
  id,
  COALESCE(customer_id::integer, 0) AS customer_id,
  COALESCE(NULLIF(category, '')::text, 'Other') AS category,
  COALESCE(NULLIF(status, '-')::text, 'Resolved') AS status,
  COALESCE(creation_date::date, '2023-01-01'::date) AS creation_date,
  COALESCE(response_time::integer, 0) AS response_time,
  ROUND(
    COALESCE(
      NULLIF(
        REGEXP_REPLACE(resolution_time::text, '[^\d.]', '', 'g'),
        ''
      )::numeric,
      0
    ),
    2
  ) AS resolution_time
FROM Support;





id  customer_id  ... response_time resolution_time
0        1         1062  ...             6            0.00
1        2          892  ...             3            0.00
2        3          433  ...             1            0.00
3        6          764  ...             3            0.00
4        7         1144  ...             2            0.00
...    ...          ...  ...           ...             ...
1982  2995         1091  ...             3            3.93
1983  2996         1024  ...             1            0.00
1984  2997         1105  ...             5            0.00
1985  2998         1608  ...             3            0.00
1986  3000         1087  ...             8            0.00

[1987 rows x 7 columns]

# Task 2

It is suspected that the response time to tickets is a big factor in unhappiness.

Calculate the minimum and maximum response time for each category of support ticket. 

Your output should include the columns `category`, `min_response` and `max_response`. 

Values should be rounded to two decimal places where appropriate. 

In [76]:
-- Write your query for task 2 in this cell
SELECT category,
       MIN(response_time) AS min_response,
       MAX(response_time) AS MAX_response
FROM support
GROUP BY category;

category  min_response  max_response
0                 Other             1             5
1                   Bug             1            13
2              Feedback             1             2
3       Billing enquiry             2             8
4  Installation Problem             5            17

# Task 3

The support team want to know more about the `rating` provided by customers who reported `Bugs` or `Installation Problem`s. 

Write a query to return the `rating` from the survey, the `customer_id`, `category` and `response_time` of the support ticket, for the customers & categories of interest. 

Use the original support table, not the output of task 1. 

In [77]:
-- Write your query for task 3 in this cell
SELECT s.rating, s.customer_id, t.category, t.response_time
FROM Survey s
JOIN Support t
  ON s.customer_id = t.customer_id
WHERE t.category IN ('Bug', 'Installation Problem');

rating  customer_id              category  response_time
0         6          621  Installation Problem              7
1         5         1703  Installation Problem              6
2         5          766  Installation Problem              7
3         5         1824                   Bug              3
4         4          931  Installation Problem              9
..      ...          ...                   ...            ...
118       4         1371  Installation Problem              6
119       4         1434  Installation Problem              6
120       3         1228                   Bug              4
121       3          926  Installation Problem              6
122       6         1227  Installation Problem              5

[123 rows x 4 columns]